In [21]:
# The objective of this file is to provide the transform_GMMC function  

# def transform_GMMC(data: pd.DataFrame) -> pd.DataFrame:
#     """Transform the dataset into the desired structure and filters."""
#     logging.info(f"Transforming data, initial size: {data.shape}")
#     # Example transformation: adding more realistic transformations
#     # This is a placeholder. Actual transformations depend on the specific needs.
#     df = data
#     logging.info(f"Transformed data size: {df.shape}")
#     return df

In [22]:
from etl import extract_csv
import pandas as pd

path='data_storage/GMMC/raw_data/GMMC-2020-M.csv'
total_path='../'+path
df= extract_csv(total_path)

2024-06-06 16:32:26,678 - INFO - Extracting data from CSV at ../data_storage/GMMC/raw_data/GMMC-2020-M.csv


In [23]:
df.head(3)
# How to show ful length of the columns
pd.set_option('display.max_columns', None)
df.head(3)
df.shape

(19044, 17)

In [24]:
# Rename columns for clarity
df.rename(columns={
    '@id': 'ID',
    'sample.samplingPoint': 'Sampling Point',
    'sample.samplingPoint.notation': 'Sampling Point Notation',
    'sample.samplingPoint.label': 'Sampling Point Label',
    'sample.sampleDateTime': 'Sample Date and Time',
    'determinand.label': 'Determinand Label',
    'determinand.definition': 'Determinand Definition',
    'determinand.notation': 'Determinand Notation',
    'resultQualifier.notation': 'Result Qualifier Notation',
    'result': 'Result',
    'codedResultInterpretation.interpretation': 'Result Interpretation',
    'determinand.unit.label': 'Unit',
    'sample.sampledMaterialType.label': 'Sample Material Type',
    'sample.isComplianceSample': 'Is Compliance Sample',
    'sample.purpose.label': 'Sample Purpose',
    'sample.samplingPoint.easting': 'Easting',
    'sample.samplingPoint.northing': 'Northing'
}, inplace=True)

df.head(3)

,ID,Sampling Point,Sampling Point Notation,Sampling Point Label,Sample Date and Time,Determinand Label,Determinand Definition,Determinand Notation,Result Qualifier Notation,Result,Result Interpretation,Unit,Sample Material Type,Is Compliance Sample,Sample Purpose,Easting,Northing
0,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NW-1086,STAGG BROOK U/S WHITCHURCH STW,2020-01-17T12:37:00,Sld Sus@105C,"Solids, Suspended at 105 C",135,NaN,43.00,NaN,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,351720,341610
1,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NW-1086,STAGG BROOK U/S WHITCHURCH STW,2020-01-17T12:37:00,Ammonia(N),Ammoniacal Nitrogen as N,111,NaN,0.16,NaN,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,351720,341610
2,http://environment.data.gov.uk/water-quality/d...,http://environment.data.gov.uk/water-quality/i...,NW-1086,STAGG BROOK U/S WHITCHURCH STW,2020-01-17T12:37:00,Chloride Ion,Chloride,172,NaN,29.00,NaN,mg/l,RIVER / RUNNING SURFACE WATER,False,ENVIRONMENTAL MONITORING STATUTORY (EU DIRECTI...,351720,341610


In [25]:
# determine the number of missing values in each column
missing_values = df.isnull().sum()
missing_values

ID                               0
Sampling Point                   0
Sampling Point Notation          0
Sampling Point Label             0
Sample Date and Time             0
Determinand Label                0
Determinand Definition           0
Determinand Notation             0
Result Qualifier Notation    16577
Result                           0
Result Interpretation        19044
Unit                             0
Sample Material Type             0
Is Compliance Sample             0
Sample Purpose                   0
Easting                          0
Northing                         0
dtype: int64

In [26]:
# drop Result Interpretation column
df.drop(columns=['Result Interpretation','Result Qualifier Notation'], inplace=True)
# determine the number of missing values in each column
missing_values = df.isnull().sum()
missing_values

ID                         0
Sampling Point             0
Sampling Point Notation    0
Sampling Point Label       0
Sample Date and Time       0
Determinand Label          0
Determinand Definition     0
Determinand Notation       0
Result                     0
Unit                       0
Sample Material Type       0
Is Compliance Sample       0
Sample Purpose             0
Easting                    0
Northing                   0
dtype: int64

In [30]:
features = ['ID', 'Sampling Point', 'Sampling Point Notation', 'Sampling Point Label', 'Sample Date and Time', 'Determinand Label', 'Determinand Definition', 'Determinand Notation', 'Result Qualifier Notation', 'Result', 'Result Interpretation', 'Unit', 'Sample Material Type', 'Is Compliance Sample', 'Sample Purpose', 'Easting', 'Northing']

# get numerical columns
numerical_columns =['Determinand Notation', 'Result', 'Easting',
       'Northing']

# Get text columns those that are not numerical
text_columns = [col for col in features if col not in numerical_columns]
text_columns

['ID',
 'Sampling Point',
 'Sampling Point Notation',
 'Sampling Point Label',
 'Sample Date and Time',
 'Determinand Label',
 'Determinand Definition',
 'Result Qualifier Notation',
 'Unit',
 'Sample Material Type',
 'Is Compliance Sample',
 'Sample Purpose']

The column 'Sampling Point' is the same as 'Sampling Point Notation' e.g. http://environment.data.gov.uk/water-quality/id/sampling-point/NW-1086
and NW-1086

In [31]:
df.drop(columns=['Sampling Point'], inplace=True)


In [32]:
# df[text_columns].head(3)
df[['ID']].head(3)
#

,ID
0,http://environment.data.gov.uk/water-quality/d...
1,http://environment.data.gov.uk/water-quality/d...
2,http://environment.data.gov.uk/water-quality/d...


In [33]:

# In id we have http://environment.data.gov.uk/water-quality/data/measurement/NW-5286182-0135
# We need to extract the last part of the string
df['ID'] = df['ID'].str.split('/').str[-1]
df[['ID']].head(3)

,ID
0,NW-5286182-0135
1,NW-5286182-0111
2,NW-5286182-0172


In [13]:
df[numerical_columns].head(3)

,Determinand Notation,Result,Result Interpretation,Easting,Northing
0,135,43.00,NaN,351720,341610
1,111,0.16,NaN,351720,341610
2,172,29.00,NaN,351720,341610


In [35]:
df[['Sample Date and Time']].head(3)

,Sample Date and Time
0,2020-01-17T12:37:00
1,2020-01-17T12:37:00
2,2020-01-17T12:37:00


In [36]:

# Convert the Sample Date and Time to datetime
df['Sample Date and Time'] = pd.to_datetime(df['Sample Date and Time'])
df[['Sample Date and Time']].head(3)


,Sample Date and Time
0,2020-01-17 12:37:00
1,2020-01-17 12:37:00
2,2020-01-17 12:37:00
